In [30]:
import os
import numpy as np
import pandas as pd
import tqdm
from ast import literal_eval
import json

In [31]:
train = pd.DataFrame()
tr_image_names = os.listdir("/mnt/hdd1/wearly/compatibility_rec/data/images/train")
train["image_name"] = tr_image_names
train.head(3)

,image_name
0,romantic_997778_jean.jpg
1,street_972741_pants.jpg
2,resort_1333785_blouse.jpg


In [32]:
styles = [strings.split("_")[0] for strings in tr_image_names]
category = [strings.split("_")[-1][:-4] for strings in tr_image_names]
train["main_style"] = styles
train["sub_category"] = category
train.head(3)

,image_name,main_style,sub_category
0,romantic_997778_jean.jpg,romantic,jean
1,street_972741_pants.jpg,street,pants
2,resort_1333785_blouse.jpg,resort,blouse


In [34]:
json_files = os.listdir("/mnt/hdd1/wearly/compatibility_rec/data/labels/train/")
colors = []
img_names = []
for jf in tqdm.tqdm(json_files):
    try:
        with open(f"/mnt/hdd1/wearly/compatibility_rec/data/labels/train/{jf}", 'r') as f:
            json_data = json.load(f)
            if len(json_data["Img"]["color"]) > 0:
                img_names.append(json_data["Img"]["imageName"])
                colors.append(json_data["Img"]["color"][0])
            else:
                img_names.append(json_data["Img"]["imageName"])
                colors.append('etc')
    except:
        pass

100%|██████████████████████████████████████████████████████████| 1457671/1457671 [10:28:19<00:00, 38.67it/s]


In [35]:
tr_dt = pd.DataFrame(columns=["image_name", "colors"])
tr_dt["image_name"] = img_names
tr_dt["colors"] = colors
tr_data = pd.merge(tr_dt, train, on="image_name")
tr_data.to_csv("tr_separ_df.csv")

In [36]:
valid = pd.DataFrame()
val_image_names = os.listdir("/mnt/hdd1/wearly/compatibility_rec/data/images/val")
valid["image_name"] = val_image_names
valid.head(3)

,image_name
0,romantic_268826_skirt.jpg
1,street_940506_knitwear.jpg
2,street_1127317_t-shirt.jpg


In [37]:
styles = [strings.split("_")[0] for strings in val_image_names]
category = [strings.split("_")[-1][:-4] for strings in val_image_names]
valid["main_style"] = styles
valid["sub_category"] = category
valid.head(3)

,image_name,main_style,sub_category
0,romantic_268826_skirt.jpg,romantic,skirt
1,street_940506_knitwear.jpg,street,knitwear
2,street_1127317_t-shirt.jpg,street,t-shirt


In [38]:
json_files = os.listdir("/mnt/hdd1/wearly/compatibility_rec/data/labels/val/")
colors = []
img_names = []
for jf in tqdm.tqdm(json_files):
    try:
        with open(f"/mnt/hdd1/wearly/compatibility_rec/data/labels/val/{jf}", 'r') as f:
            json_data = json.load(f)
            if len(json_data["Img"]["color"]) > 0:
                img_names.append(json_data["Img"]["imageName"])
                colors.append(json_data["Img"]["color"][0])
            else:
                img_names.append(json_data["Img"]["imageName"])
                colors.append('etc')
    except:
        pass

100%|███████████████████████████████████████████████████████████████| 182315/182315 [52:23<00:00, 58.00it/s]


In [39]:
val_dt = pd.DataFrame(columns=["image_name", "colors"])
val_dt["image_name"] = img_names
val_dt["colors"] = colors
val_data = pd.merge(val_dt, valid, on="image_name")
val_data.to_csv("val_separ_df.csv")

# add mete data

In [42]:
import pymysql as pymysql

In [43]:
conn = pymysql.connect(host='localhost',
                           port=3306,
                           user='root',
                           password='wearly123',
                           db='kfashion_DB', charset='utf8mb4')
curs = conn.cursor()
try:
    curs = conn.cursor()
    sql = """select * from kfashion"""
    curs.execute(sql)
    result = curs.fetchall()
finally:
    conn.close()

In [44]:
sql_df = pd.DataFrame(result, columns=['image_name', 'main_category', 'sub_category', 'style', 'text'])
sql_df

,image_name,main_category,sub_category,style,text
0,feminine_1204621.jpg,원피스,드레스,feminine,"{'기장': '미니', '색상': '베이지', '카테고리': '드레스', '디테일'..."
1,romantic_292213.jpg,하의,스커트,romantic,"{'기장': '발목', '색상': '화이트', '카테고리': '스커트', '디테일'..."
2,romantic_987357.jpg,상의,블라우스,romantic,"{'색상': '와인', '서브색상': '퍼플', '카테고리': '블라우스', '디테..."
3,romantic_987357.jpg,하의,스커트,romantic,"{'기장': '미니', '색상': '화이트', '카테고리': '스커트', '소재':..."
4,feminine_1099505.jpg,원피스,드레스,feminine,"{'기장': '미니', '색상': '블랙', '서브색상': '화이트', '카테고리'..."
...,...,...,...,...,...
1531333,resort_1241663.jpg,원피스,드레스,resort,"{'기장': '미디', '색상': '실버', '카테고리': '드레스', '디테일':..."
1531334,modern_1053307.jpg,상의,블라우스,modern,"{'기장': '노멀', '색상': '핑크', '카테고리': '블라우스', '디테일'..."
1531335,modern_1053307.jpg,하의,팬츠,modern,"{'기장': '발목', '색상': '화이트', '카테고리': '팬츠', '소재': ..."
1531336,street_1179713.jpg,상의,셔츠,street,"{'기장': '롱', '색상': '화이트', '서브색상': '블랙', '카테고리':..."


In [45]:
sql_df.loc[sql_df.sub_category == '탑', 'sub_category'] = 'tops'
sql_df.loc[sql_df.sub_category == '블라우스', 'sub_category'] = 'blouse'
sql_df.loc[sql_df.sub_category == '티셔츠', 'sub_category'] = 't-shirt'
sql_df.loc[sql_df.sub_category == '니트웨어', 'sub_category'] = 'knitwear'        
sql_df.loc[sql_df.sub_category == '셔츠', 'sub_category'] = 'shirt'
sql_df.loc[sql_df.sub_category == '브라탑', 'sub_category'] = 'bratop'
sql_df.loc[sql_df.sub_category == '후드티', 'sub_category'] = 'hoody'
sql_df.loc[sql_df.sub_category == '청바지', 'sub_category'] = 'jean'
sql_df.loc[sql_df.sub_category == '팬츠', 'sub_category'] = 'pants'
sql_df.loc[sql_df.sub_category == '스커트', 'sub_category'] = 'skirt'
sql_df.loc[sql_df.sub_category == '레깅스', 'sub_category'] = 'leggings'
sql_df.loc[sql_df.sub_category == '조거팬츠', 'sub_category'] = 'jogger pants'
sql_df.loc[sql_df.sub_category == '코트', 'sub_category'] = 'coats'
sql_df.loc[sql_df.sub_category == '재킷', 'sub_category'] = 'jacket'
sql_df.loc[sql_df.sub_category == '점퍼', 'sub_category'] = 'jumper'
sql_df.loc[sql_df.sub_category == '패딩', 'sub_category'] = 'padded jacket'
sql_df.loc[sql_df.sub_category == '베스트', 'sub_category'] = 'best'
sql_df.loc[sql_df.sub_category == '가디건', 'sub_category'] = 'cardigan'
sql_df.loc[sql_df.sub_category == '짚업', 'sub_category'] = 'zipup'
sql_df.loc[sql_df.sub_category == '드레스' ,'sub_category'] = 'dress'
sql_df.loc[sql_df.sub_category == '점프수트' ,'sub_category'] = 'jumpsuit'
sql_df.loc[sql_df.sub_category == '래깅스' ,'sub_category'] = 'leggings' 

In [46]:
s_n = [sql_df.image_name[i][:-4] + '_' + sql_df.sub_category[i] + '.jpg' for i in tqdm.tqdm(range(len(sql_df)))]

100%|█████████████████████████████████████████████████████████| 1531338/1531338 [00:13<00:00, 110724.92it/s]


In [47]:
sql_df["image_name"] = s_n

In [49]:
tr_d = pd.merge(tr_data, sql_df , on = 'image_name')
tr_d

,image_name,colors,main_style,sub_category_x,main_category,sub_category_y,style,text
0,street_1330115_t-shirt.jpg,lightgray,street,t-shirt,상의,t-shirt,street,"{'기장': '노멀', '색상': '화이트', '서브색상': '블랙', '카테고리'..."
1,romantic_1168617_jean.jpg,darkgray,romantic,jean,하의,jean,romantic,"{'기장': '미니', '색상': '블루', '카테고리': '청바지', '디테일':..."
2,romantic_789637_blouse.jpg,etc,romantic,blouse,상의,blouse,romantic,"{'기장': '노멀', '색상': '화이트', '카테고리': '블라우스', '옷깃'..."
3,sexy_786856_dress.jpg,etc,sexy,dress,원피스,dress,sexy,"{'기장': '미니', '색상': '블랙', '카테고리': '드레스', '소매기장'..."
4,street_1340234_pants.jpg,etc,street,pants,하의,pants,street,"{'기장': '미니', '색상': '화이트', '카테고리': '팬츠', '소재': ..."
...,...,...,...,...,...,...,...,...
1204207,resort_275300_dress.jpg,darkgray,resort,dress,원피스,dress,resort,"{'기장': '미디', '색상': '베이지', '카테고리': '드레스', '옷깃':..."
1204208,street_719801_jean.jpg,darkgray,street,jean,하의,jean,street,"{'기장': '발목', '색상': '블루', '카테고리': '청바지', '소재': ..."
1204209,sophisticated_1122504_skirt.jpg,etc,sophisticated,skirt,하의,skirt,sophisticated,"{'기장': '미니', '색상': '블랙', '서브색상': '베이지', '카테고리'..."
1204210,street_1018127_t-shirt.jpg,lavender,street,t-shirt,상의,t-shirt,street,"{'기장': '노멀', '색상': '화이트', '카테고리': '티셔츠', '디테일'..."


In [50]:
val_d = pd.merge(val_data, sql_df , on = 'image_name')
val_d

,image_name,colors,main_style,sub_category_x,main_category,sub_category_y,style,text
0,street_411249_shirt.jpg,brown,street,shirt,상의,shirt,street,"{'기장': '노멀', '색상': '브라운', '서브색상': '화이트', '카테고리..."
1,resort_1030135_jacket.jpg,gray,resort,jacket,아우터,jacket,resort,"{'기장': '하프', '색상': '그레이', '카테고리': '재킷', '옷깃': ..."
2,street_724446_jean.jpg,black,street,jean,하의,jean,street,"{'기장': '발목', '색상': '블루', '카테고리': '청바지', '소재': ..."
3,sophisticated_1102883_pants.jpg,darkgray,sophisticated,pants,하의,pants,sophisticated,"{'기장': '발목', '색상': '베이지', '카테고리': '팬츠', '소재': ..."
4,classic_776968_knitwear.jpg,lightgray,classic,knitwear,상의,knitwear,classic,"{'기장': '노멀', '색상': '화이트', '카테고리': '니트웨어', '소매기..."
...,...,...,...,...,...,...,...,...
149143,street_946637_t-shirt.jpg,etc,street,t-shirt,상의,t-shirt,street,"{'기장': '노멀', '색상': '와인', '서브색상': '오렌지', '카테고리'..."
149144,street_1011863_jacket.jpg,brown,street,jacket,아우터,jacket,street,"{'기장': '노말', '색상': '옐로우', '서브색상': '화이트', '카테고리..."
149145,feminine_974300_dress.jpg,gray,feminine,dress,원피스,dress,feminine,"{'기장': '니렝스', '색상': '와인', '카테고리': '드레스', '디테일'..."
149146,romantic_1216894_skirt.jpg,darkgray,romantic,skirt,하의,skirt,romantic,"{'기장': '미디', '색상': '카키', '카테고리': '스커트', '디테일':..."


In [56]:
# str to dict
tr_d['text'] = [literal_eval(tr_d['text'][i]) for i in tqdm.tqdm(range(len(tr_d)))]
val_d['text'] = [literal_eval(val_d['text'][i]) for i in tqdm.tqdm(range(len(val_d)))]

100%|████████████████████████████████████████████████████████████| 149148/149148 [00:05<00:00, 28261.26it/s]


In [57]:
tr_d['txt_keys'] = [list(tr_d['text'][i].keys()) for i in tqdm.tqdm(range(len(tr_d)))]
tr_d['txt_values'] = [list(tr_d['text'][i].values()) for i in tqdm.tqdm(range(len(tr_d)))]

100%|█████████████████████████████████████████████████████████| 1204212/1204212 [00:10<00:00, 118265.06it/s]


In [58]:
val_d['txt_keys'] = [list(val_d['text'][i].keys()) for i in tqdm.tqdm(range(len(val_d)))]
val_d['txt_values'] = [list(val_d['text'][i].values()) for i in tqdm.tqdm(range(len(val_d)))]

100%|███████████████████████████████████████████████████████████| 149148/149148 [00:00<00:00, 236110.54it/s]


In [59]:
tr_d['length'] = [len(tr_d.txt_keys[i]) for i in tqdm.tqdm(range(len(tr_d)))]
val_d['length'] = [len(val_d.txt_keys[i]) for i in tqdm.tqdm(range(len(val_d)))]

100%|███████████████████████████████████████████████████████████| 149148/149148 [00:00<00:00, 205446.03it/s]


In [88]:
def make_meta_col(dt):
    df = dt.copy()
    fit_ls = []
    for idx,i in enumerate(df.text):
        try:
            if i['핏'] is not None:
                fit_ls.append((idx,i['핏']))
        except:
            pass

    print_ls = []
    for idx,i in enumerate(df.text):
        try:
            if i['프린트'] is not None:
                print_ls.append((idx,i['프린트']))
        except:
            pass

    color_ls = []
    for idx,i in enumerate(df.text):
        try:
            if i['색상'] is not None:
                color_ls.append((idx,i['색상']))
        except:
            pass
        
    
    color_ids = [x[0] for x in color_ls]
    print_ids = [x[0] for x in print_ls]
    fit_ids = [x[0] for x in fit_ls]
    ids = list(set(color_ids)&set(print_ids)&set(fit_ids)) #final idx
    df_t = df.iloc[ids,:]
    
    fit_ls = []
    print_ls = []
    color_ls = []
    for i in df_t['text']:
        if i['프린트'] is not None:
            print_ls.append(i['프린트'])
        if i['색상'] is not None:
            color_ls.append(i['색상'])
        if i['핏'] is not None:
            fit_ls.append(i['핏'])
        else:
            pass
    df_t['color'] = color_ls
    df_t['fit'] = fit_ls
    df_t['print'] = print_ls
    df_t = df_t[['image_name', 'colors', 'color', 'sub_category_x', 'sub_category_y', 'style', 'fit', 'print']]
    
    print_ls = [len(x) for x in df_t.print]
    df_t['print_length'] = print_ls
    df_t = df_t.loc[df_t['print_length'] == 1, :]
    
    df_t = df_t.reset_index(drop=True)
    return df_t

In [89]:
f_tr = make_meta_col(dt=tr_d)
f_val = make_meta_col(dt=val_d)

/tmp/ipykernel_3438793/2790657255.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_t['color'] = color_ls
/tmp/ipykernel_3438793/2790657255.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_t['fit'] = fit_ls
/tmp/ipykernel_3438793/2790657255.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing

In [91]:
f_tr.to_csv('f_tr.csv')
f_val.to_csv('f_val.csv')